## 问题定义

本问题为求解三维Taylor Green涡流。


### 求解目标

假定雷诺数为500，要求求解边长为$2\pi$，边界条件为周期边界条件并且具有如下初始条件的Taylor Green涡流。

![IC](./resource/IC_eq.png)

## Moving Time Window

移动窗口法本质上是通过将长时间拆解为多个短时序进行顺序求解从而缓解PINN在长时序求解中的不足。

示意图如下：

![Moving Time Window](./resource/MTW.png)

对于每一个Time Window，前面的求解结果会被作为新的初始条件监督后面的训练，每一个Window也是重新初始化的（存在误差累积的问题）

## Causal Training

结合使用Causal Trainin的Moving Time Window。对于Causal Training的描述参考08。在本例中，对于每一个Time Window，均使用Causal Loss保证序列的因果性


## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"
os.environ["PYTORCH_JIT"]="0"  # 需要手动禁用JIT，否则MovingTimeWindowArch会报错

In [2]:
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose

import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np

from sympy import Symbol, Eq, Abs, sin, cos

import modulus.sym
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.utils.io import csv_to_dict
from modulus.sym.solver import SequentialSolver
from modulus.sym.domain import Domain

from modulus.sym.loss.loss import CausalLossNorm
from modulus.sym.geometry.parameterization import OrderedParameterization

from modulus.sym.geometry.primitives_3d import Box

from modulus.sym.models.fully_connected import FullyConnectedArch
from modulus.sym.models.moving_time_window import MovingTimeWindowArch
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.sym.domain.inferencer import PointVTKInferencer
from modulus.sym.utils.io import (
    VTKUniformGrid,
)
from modulus.sym.key import Key
from modulus.sym.node import Node
from modulus.sym.eq.pdes.navier_stokes import NavierStokes

In [3]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 300000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: ${training.rec_results_freq}
  rec_inference_freq: ${training.rec_results_freq}
  rec_monitor_freq: ${training.rec_results_freq}
  rec_constraint_freq: 5000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: false
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arc

### 定义必要组件

#### PDE

In [4]:
ns = NavierStokes(nu=0.002, rho=1.0, dim=3, time=True)

#### Time Window Params

In [5]:
time_window_size = 1.0  # 窗口宽度
t_symbol = Symbol("t")
time_range = {t_symbol: (0, time_window_size)}
nr_time_windows = 10  # 窗口数量

#### Geo

In [6]:
# define sympy variables to parametrize domain curves
x, y, z = Symbol("x"), Symbol("y"), Symbol("z")

# make geometry for problem
channel_length = (0.0, 2 * np.pi)
channel_width = (0.0, 2 * np.pi)
channel_height = (0.0, 2 * np.pi)
box_bounds = {x: channel_length, y: channel_width, z: channel_height}

# define geometry
rec = Box(
    (channel_length[0], channel_width[0], channel_height[0]),
    (channel_length[1], channel_width[1], channel_height[1]),
    parameterization=OrderedParameterization(time_range, key=t_symbol),  # 序列顺序化
)

#### Model

In [7]:

# 定义简单的全连接网络
# 输入为空间坐标和时间
# 输出为三个方向速度以及压强
flow_net = FullyConnectedArch(
    input_keys=[Key("x"), Key("y"), Key("z"), Key("t")],
    output_keys=[Key("u"), Key("v"), Key("w"), Key("p")],
    periodicity={"x": channel_length, "y": channel_width, "z": channel_height},  # 周期边界，允许模型进行周期性预测
    layer_size=256,
)
time_window_net = MovingTimeWindowArch(flow_net, time_window_size)  # 定义MTW

#### Node

In [8]:
nodes = ns.make_nodes() + [time_window_net.make_node(name="time_window_network")]

#### Domain

在Domain中定义约束以及训练所需的各种组件

在本例中分别声明IC的Domain和MTW的Domain。原因在于对于不同的Window，IC是一直变化的

In [9]:
# make initial condition domain
ic_domain = Domain("initial_conditions")

# make moving window domain
window_domain = Domain("window")

#### 初始条件

In [10]:
# make initial condition
# 真实初始条件，第一个Window
ic = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={
        "u": sin(x) * cos(y) * cos(z),
        "v": -cos(x) * sin(y) * cos(z),
        "w": 0,
        "p": 1.0 / 16 * (cos(2 * x) + cos(2 * y)) * (cos(2 * z) + 2),
    },
    batch_size=cfg.batch_size.initial_condition,
    bounds=box_bounds,
    lambda_weighting={"u": 100, "v": 100, "w": 100, "p": 100},
    parameterization={t_symbol: 0},  # 覆盖序列顺序化对t的约束
)
ic_domain.add_constraint(ic, name="ic")


# make constraint for matching previous windows initial condition
# 对于后续每一个Window，其初始条件是上一个Window预测的结果
ic = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u_prev_step_diff": 0, "v_prev_step_diff": 0, "w_prev_step_diff": 0},
    batch_size=cfg.batch_size.interior,
    bounds=box_bounds,
    lambda_weighting={
        "u_prev_step_diff": 100,
        "v_prev_step_diff": 100,
        "w_prev_step_diff": 100,
    },
    parameterization={t_symbol: 0},  # 时间上也重置为0，覆盖序列顺序化对t的约束
)
window_domain.add_constraint(ic, name="ic")

#### PDE约束

In [11]:
# make interior constraint
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"continuity": 0, "momentum_x": 0, "momentum_y": 0, "momentum_z": 0},
    bounds=box_bounds,
    batch_size=4000,
    # parameterization=time_range,
    loss=CausalLossNorm(eps=1.0),  # 引入CausalLoss
    fixed_dataset=False,
    shuffle=False,
)
ic_domain.add_constraint(interior, name="interior")
window_domain.add_constraint(interior, name="interior")

验证器以及其他必要组件

In [12]:
for i, specific_time in enumerate(np.linspace(0, time_window_size, 10)):  # 对于每一个切片均运行结果
    vtk_obj = VTKUniformGrid(
        bounds=[(0, 2 * np.pi), (0, 2 * np.pi), (0, 2 * np.pi)],
        npoints=[128, 128, 128],
        export_map={"u": ["u", "v", "w"], "p": ["p"]},
    )
    
    # 创建Inferencer
    grid_inference = PointVTKInferencer(
        vtk_obj=vtk_obj,
        nodes=nodes,
        input_vtk_map={"x": "x", "y": "y", "z": "z"},
        output_names=["u", "v", "w", "p"],
        requires_grad=False,
        invar={"t": np.full([128**3, 1], specific_time)},  # t序列
        batch_size=100000,
    )
    ic_domain.add_inferencer(grid_inference, name="time_slice_" + str(i).zfill(4))
    window_domain.add_inferencer(
        grid_inference, name="time_slice_" + str(i).zfill(4)
    )

### 求解器以及求解

In [13]:
# 定义求解器
# 这里使用序列求解器
# 不同于一般的求解器
# 序列求解器要求传入求解次数-求解域对
slv = SequentialSolver(
    cfg,
    [(1, ic_domain), (nr_time_windows, window_domain)],  # 表示： ic_domain运行一次，window_domain运行nr_time_windows次
    custom_update_operation=time_window_net.move_window,  # 表示更新参数
)

手动加载日志系统

In [14]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [15]:
slv.solve()

2024-02-28 03:27:02,714 - INFO - Solving for Domain initial_conditions, iteration 0
2024-02-28 03:27:03,129 - INFO - attempting to restore from: /workspace/13_3D_TaylorGreenFlow_Causal/outputs/initial_conditions
2024-02-28 03:27:03,130 - WARNING - optimizer checkpoint not found
2024-02-28 03:27:03,131 - WARNING - model time_window_network.0.pth not found
2024-02-28 03:27:04,605 - INFO - [step:          0] record constraint batch time:  2.457e-01s
2024-02-28 03:27:46,826 - INFO - [step:          0] record inferencers time:  4.222e+01s
2024-02-28 03:27:46,846 - INFO - [step:          0] saved checkpoint to /workspace/13_3D_TaylorGreenFlow_Causal/outputs/initial_conditions
2024-02-28 03:27:46,849 - INFO - [step:          0] loss:  6.645e+03
2024-02-28 03:27:49,943 - INFO - Attempting cuda graph building, this may take a bit...
2024-02-28 03:28:04,493 - INFO - [step:        100] loss:  5.294e+01, time/iteration:  1.764e+02 ms
2024-02-28 03:28:09,214 - INFO - [step:        200] loss:  3.593

2024-02-28 03:37:51,528 - INFO - [step:       6000] loss:  2.561e+00, time/iteration:  4.856e+02 ms
2024-02-28 03:37:57,970 - INFO - [step:       6100] loss:  3.058e+00, time/iteration:  6.442e+01 ms
2024-02-28 03:38:04,387 - INFO - [step:       6200] loss:  3.658e+00, time/iteration:  6.415e+01 ms
2024-02-28 03:38:14,997 - INFO - [step:       6300] loss:  2.774e+00, time/iteration:  1.061e+02 ms
2024-02-28 03:38:21,505 - INFO - [step:       6400] loss:  3.076e+00, time/iteration:  6.506e+01 ms
2024-02-28 03:38:26,522 - INFO - [step:       6500] loss:  6.157e+00, time/iteration:  5.014e+01 ms
2024-02-28 03:38:31,155 - INFO - [step:       6600] loss:  5.044e+00, time/iteration:  4.632e+01 ms
2024-02-28 03:38:35,760 - INFO - [step:       6700] loss:  2.149e+00, time/iteration:  4.603e+01 ms
2024-02-28 03:38:40,639 - INFO - [step:       6800] loss:  4.240e+00, time/iteration:  4.877e+01 ms
2024-02-28 03:38:45,603 - INFO - [step:       6900] loss:  2.288e+00, time/iteration:  4.963e+01 ms


2024-02-28 03:48:46,291 - INFO - [step:      12800] loss:  8.294e-01, time/iteration:  7.910e+01 ms
2024-02-28 03:48:54,068 - INFO - [step:      12900] loss:  6.640e-01, time/iteration:  7.776e+01 ms
2024-02-28 03:49:34,860 - INFO - [step:      13000] record inferencers time:  3.451e+01s
2024-02-28 03:49:34,881 - INFO - [step:      13000] saved checkpoint to /workspace/13_3D_TaylorGreenFlow_Causal/outputs/initial_conditions
2024-02-28 03:49:34,883 - INFO - [step:      13000] loss:  8.349e-01, time/iteration:  4.081e+02 ms
2024-02-28 03:49:41,139 - INFO - [step:      13100] loss:  1.011e+00, time/iteration:  6.255e+01 ms
2024-02-28 03:49:46,917 - INFO - [step:      13200] loss:  6.614e-01, time/iteration:  5.776e+01 ms
2024-02-28 03:49:52,828 - INFO - [step:      13300] loss:  1.104e+00, time/iteration:  5.910e+01 ms
2024-02-28 03:49:58,879 - INFO - [step:      13400] loss:  4.987e-01, time/iteration:  6.049e+01 ms
2024-02-28 03:50:05,534 - INFO - [step:      13500] loss:  7.879e-01, ti

2024-02-28 04:01:31,524 - INFO - [step:      19400] loss:  3.724e-01, time/iteration:  1.454e+02 ms
2024-02-28 04:01:45,703 - INFO - [step:      19500] loss:  2.346e-01, time/iteration:  1.418e+02 ms
2024-02-28 04:01:57,724 - INFO - [step:      19600] loss:  8.575e-01, time/iteration:  1.202e+02 ms
2024-02-28 04:02:08,785 - INFO - [step:      19700] loss:  2.198e-01, time/iteration:  1.106e+02 ms
2024-02-28 04:02:18,848 - INFO - [step:      19800] loss:  4.111e-01, time/iteration:  1.006e+02 ms
2024-02-28 04:02:28,909 - INFO - [step:      19900] loss:  2.718e-01, time/iteration:  1.006e+02 ms
2024-02-28 04:02:39,471 - INFO - [step:      20000] record constraint batch time:  1.540e-01s
2024-02-28 04:03:29,364 - INFO - [step:      20000] record inferencers time:  4.989e+01s
2024-02-28 04:03:29,388 - INFO - [step:      20000] saved checkpoint to /workspace/13_3D_TaylorGreenFlow_Causal/outputs/initial_conditions
2024-02-28 04:03:29,390 - INFO - [step:      20000] loss:  4.595e-01, time/ite

2024-02-28 04:19:16,586 - INFO - [step:      26000] saved checkpoint to /workspace/13_3D_TaylorGreenFlow_Causal/outputs/initial_conditions
2024-02-28 04:19:16,589 - INFO - [step:      26000] loss:  1.602e-01, time/iteration:  5.409e+02 ms
2024-02-28 04:19:32,333 - INFO - [step:      26100] loss:  4.276e-01, time/iteration:  1.574e+02 ms
2024-02-28 04:19:43,673 - INFO - [step:      26200] loss:  1.116e-01, time/iteration:  1.134e+02 ms
2024-02-28 04:19:57,181 - INFO - [step:      26300] loss:  7.060e-02, time/iteration:  1.351e+02 ms
2024-02-28 04:20:09,119 - INFO - [step:      26400] loss:  2.168e-01, time/iteration:  1.194e+02 ms
2024-02-28 04:20:19,100 - INFO - [step:      26500] loss:  1.103e-01, time/iteration:  9.979e+01 ms
2024-02-28 04:20:26,463 - INFO - [step:      26600] loss:  2.054e-01, time/iteration:  7.361e+01 ms
2024-02-28 04:20:36,105 - INFO - [step:      26700] loss:  4.613e-01, time/iteration:  9.640e+01 ms
2024-02-28 04:20:45,425 - INFO - [step:      26800] loss:  5.

...

2024-02-28 14:02:53,798 - INFO - [step:       5000] loss:  3.975e-01, time/iteration:  4.571e+02 ms
2024-02-28 14:02:59,031 - INFO - [step:       5100] loss:  1.302e-01, time/iteration:  5.232e+01 ms
2024-02-28 14:03:07,207 - INFO - [step:       5200] loss:  5.500e-01, time/iteration:  8.174e+01 ms
2024-02-28 14:03:15,599 - INFO - [step:       5300] loss:  1.833e-01, time/iteration:  8.391e+01 ms
2024-02-28 14:03:22,652 - INFO - [step:       5400] loss:  6.537e-01, time/iteration:  7.052e+01 ms
2024-02-28 14:03:28,996 - INFO - [step:       5500] loss:  7.330e-01, time/iteration:  6.342e+01 ms
2024-02-28 14:03:36,055 - INFO - [step:       5600] loss:  4.623e-01, time/iteration:  7.057e+01 ms
2024-02-28 14:03:42,382 - INFO - [step:       5700] loss:  2.311e-01, time/iteration:  6.326e+01 ms
2024-02-28 14:03:49,039 - INFO - [step:       5800] loss:  7.953e-01, time/iteration:  6.655e+01 ms
2024-02-28 14:03:55,180 - INFO - [step:       5900] loss:  1.653e-01, time/iteration:  6.139e+01 ms


2024-02-28 14:14:20,313 - INFO - [step:      11800] loss:  2.769e-01, time/iteration:  4.941e+01 ms
2024-02-28 14:14:25,265 - INFO - [step:      11900] loss:  2.334e-01, time/iteration:  4.951e+01 ms
2024-02-28 14:15:11,515 - INFO - [step:      12000] record inferencers time:  4.084e+01s
2024-02-28 14:15:11,540 - INFO - [step:      12000] saved checkpoint to /workspace/13_3D_TaylorGreenFlow_Causal/outputs/window_0000
2024-02-28 14:15:11,542 - INFO - [step:      12000] loss:  1.294e-01, time/iteration:  4.628e+02 ms
2024-02-28 14:15:18,811 - INFO - [step:      12100] loss:  1.406e-01, time/iteration:  7.269e+01 ms
2024-02-28 14:15:26,461 - INFO - [step:      12200] loss:  1.882e-01, time/iteration:  7.649e+01 ms
2024-02-28 14:15:33,754 - INFO - [step:      12300] loss:  3.282e-01, time/iteration:  7.291e+01 ms
2024-02-28 14:15:41,869 - INFO - [step:      12400] loss:  1.134e-01, time/iteration:  8.114e+01 ms
2024-02-28 14:15:49,507 - INFO - [step:      12500] loss:  3.488e-01, time/iter

2024-02-28 14:26:13,486 - INFO - [step:      18400] loss:  8.809e-02, time/iteration:  7.650e+01 ms
2024-02-28 14:26:20,541 - INFO - [step:      18500] loss:  2.055e-01, time/iteration:  7.053e+01 ms
2024-02-28 14:26:28,037 - INFO - [step:      18600] loss:  1.434e-01, time/iteration:  7.495e+01 ms
2024-02-28 14:26:35,073 - INFO - [step:      18700] loss:  1.504e-01, time/iteration:  7.034e+01 ms
2024-02-28 14:26:41,245 - INFO - [step:      18800] loss:  9.953e-02, time/iteration:  6.171e+01 ms
2024-02-28 14:26:46,256 - INFO - [step:      18900] loss:  1.778e-01, time/iteration:  5.010e+01 ms
2024-02-28 14:27:30,853 - INFO - [step:      19000] record inferencers time:  3.901e+01s
2024-02-28 14:27:30,877 - INFO - [step:      19000] saved checkpoint to /workspace/13_3D_TaylorGreenFlow_Causal/outputs/window_0000
2024-02-28 14:27:30,879 - INFO - [step:      19000] loss:  5.571e-02, time/iteration:  4.462e+02 ms
2024-02-28 14:27:39,249 - INFO - [step:      19100] loss:  1.939e-01, time/iter

2024-02-28 14:38:05,956 - INFO - [step:      25000] saved checkpoint to /workspace/13_3D_TaylorGreenFlow_Causal/outputs/window_0000
2024-02-28 14:38:05,958 - INFO - [step:      25000] loss:  1.966e-01, time/iteration:  4.696e+02 ms
2024-02-28 14:38:15,095 - INFO - [step:      25100] loss:  1.020e-01, time/iteration:  9.136e+01 ms
2024-02-28 14:38:23,635 - INFO - [step:      25200] loss:  4.815e-02, time/iteration:  8.539e+01 ms
2024-02-28 14:38:31,172 - INFO - [step:      25300] loss:  8.995e-02, time/iteration:  7.535e+01 ms
2024-02-28 14:38:38,846 - INFO - [step:      25400] loss:  4.543e-02, time/iteration:  7.672e+01 ms
2024-02-28 14:38:46,242 - INFO - [step:      25500] loss:  1.148e-01, time/iteration:  7.395e+01 ms
2024-02-28 14:38:54,123 - INFO - [step:      25600] loss:  1.305e-01, time/iteration:  7.880e+01 ms


KeyboardInterrupt: 

### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

In [ ]:
import glob
import matplotlib.pyplot as plt

In [ ]:
# get list of steps
window_dirs = glob.glob("./outputs/network_checkpoint/*")
window_dirs.sort()
window_dirs = [x for x in window_dirs if os.path.isdir(x)]

# read each file in each dir and store tke
index = 0
time_points = []
tke_points = []
for i, d in enumerate(window_dirs):
    # get list of slices
    slice_files = glob.glob(d + "/inferencers/time_slice_*.npz")
    slice_files.sort()

    for f in slice_files:
        predicted_data = np.load(f, allow_pickle=True)["arr_0"].item()

        # shift t
        predicted_data["t"] += i
        if float(predicted_data["t"][0, 0, 0]) < 10.0:
            # store time
            time_points.append(float(predicted_data["t"][0, 0, 0]))

            # compute tke and store
            tke = np.mean(
                predicted_data["u"] ** 2 / 2
                + predicted_data["v"] ** 2 / 2
                + predicted_data["w"] ** 2 / 2
            )
            tke_points.append(tke)
            index += 1
tke_points = tke_points / np.max(tke_points)

In [ ]:
# load validation tke data
file_path = "openfoam/validation_tke"
if os.path.exists(to_absolute_path(file_path)):
    validation_tke_128 = csv_to_dict("openfoam/validation_tke/tke_mean_Re500_N128.csv")
    validation_tke_256 = csv_to_dict("openfoam/validation_tke/tke_mean_Re500_N256.csv")

    plt.plot(
        validation_tke_128["Time"][:, 0],
        validation_tke_128["TKE_mean"][:, 0],
        label="Spectral Solver (grid res: 128)",
    )
    plt.plot(
        validation_tke_256["Time"][:, 0],
        validation_tke_256["TKE_mean"][:, 0],
        label="Spectral Solver (grid res: 256)",
    )
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

# plot turbulent kinetic energy decay
plt.plot(time_points, tke_points, label="Modulus")

plt.legend()
plt.title("TKE")
plt.ylabel("TKE")
plt.xlabel("time")
plt.show()